<a href="https://colab.research.google.com/github/drbearcub/coref_resolution_training/blob/main/New_new_new_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install trl
!pip install Dataset
!pip install json

from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from trl import SFTConfig, SFTTrainer, DataCollatorForCompletionOnlyLM
import json
import random

from datasets import load_dataset

from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, EarlyStoppingCallback

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"



ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:

file_path = '/content/drive/MyDrive/Colab Notebooks/restructured.jsonl'
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")
#tokenizer.padding_side = 'left'
#tokenizer.pad_token = tokenizer.eos_token

# Load JSONL into a list of JSON objects
data = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))

#MAX_CHARS_FOR_PAST_CHAT = 5000 # ~ 1500 tokens
MAX_CHARS_FOR_PAST_CHAT = 5000 # ~ 1500 tokens

def data_transform_func(example):
    course_str = example['course']
    user_query_str = example['UserQuery']
    resolved_query_str = example['ResolvedQuery']
    past_chat_str = example['pastChat']

    # Trim from the beginning if too long
    if len(past_chat_str) > MAX_CHARS_FOR_PAST_CHAT:
        past_chat_str = past_chat_str[-MAX_CHARS_FOR_PAST_CHAT:]

    prompt =  f"[Course]{course_str}\n[PastChat]{past_chat_str}\n[UserQuery]{user_query_str}"

    return {"prompt": prompt, "completion": resolved_query_str}


random.shuffle(data)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/685 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/644 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/441 [00:00<?, ?B/s]

In [5]:
train_dataSet_processed = list(map(data_transform_func, data))
print(train_dataSet_processed[0])
print(len(train_dataSet_processed))


#save as jsonl
output_file_path = '/content/drive/MyDrive/Colab Notebooks/restructured_processed.jsonl'
with open(output_file_path, 'w', encoding='utf-8') as f:
    for item in train_dataSet_processed:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')



{'prompt': '[Course]KBAI\n[PastChat]None\n[UserQuery]policy for late submission', 'completion': 'policy for late submission'}
4511


In [6]:
# load jsonl dataset
loaded_dataset = load_dataset("json", data_files="/content/drive/MyDrive/Colab Notebooks/restructured_processed.jsonl", split="train")
# Split the dataset into training and evaluation sets (e.g., 90% train, 10% eval)
train_test_split = loaded_dataset.train_test_split(test_size=0.1)
train_dataset = train_test_split['train']
eval_dataset = train_test_split['test']

Generating train split: 0 examples [00:00, ? examples/s]

In [7]:
output_file_path = '/content/drive/MyDrive/Colab Notebooks/restructured_processed_train.jsonl'
with open(output_file_path, 'w', encoding='utf-8') as f:
    for item in train_dataset:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')

output_file_path = '/content/drive/MyDrive/Colab Notebooks/restructured_processed_test.jsonl'
with open(output_file_path, 'w', encoding='utf-8') as f:
    for item in eval_dataset:
        json.dump(item, f, ensure_ascii=False)
        f.write('\n')



In [8]:
# load jsonl dataset
train_dataset = load_dataset("json", data_files="/content/drive/MyDrive/Colab Notebooks/restructured_processed_train.jsonl", split="train")
eval_dataset = load_dataset("json", data_files="/content/drive/MyDrive/Colab Notebooks/restructured_processed_test.jsonl", split="train")


Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [13]:
del model

NameError: name 'model' is not defined

In [ ]:
import torch
from trl import DataCollatorForCompletionOnlyLM


#model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen3-0.6B", torch_dtype=torch.bfloat16)
model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")

training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=14,
    gradient_accumulation_steps=6,

    max_steps=100000,
    logging_steps=1,
    report_to=[],
    #bf16=True,

    # --- Early Stopping and Evaluation Arguments ---
    eval_strategy="steps",
    eval_steps=20,
    save_strategy="steps",
    save_steps=240,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,

    #optim="paged_adamw_32bit",
    # learning_rate=2e-4,
    # lr_scheduler_type="cosine",
    # warmup_ratio=0.03,
    # fp16=True,
)

#load train_dataSet_processed into model
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset, # Pass the evaluation dataset
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10, early_stopping_threshold=0.0)], # Add the callback here
)

trainer.train()

pytorch_model.bin:   0%|          | 0.00/663M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662M [00:00<?, ?B/s]

Converting train dataset to ChatML:   0%|          | 0/4059 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/4059 [00:00<?, ? examples/s]

...

Converting eval dataset to ChatML:   0%|          | 0/452 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/452 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/452 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/452 [00:00<?, ? examples/s]

Step,Training Loss,Validation Loss
20,0.297400,0.263025


In [9]:
output_directory = "/content/drive/MyDrive/Colab Notebooks/overfitted_model"
trainer.save_model(output_directory)

In [10]:
#load model from drive
from transformers import pipeline
model = AutoModelForCausalLM.from_pretrained("/content/drive/MyDrive/Colab Notebooks/overfitted_model")
tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/overfitted_model")
generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

Device set to use cuda:0


In [11]:
import transformers
from tqdm import tqdm

transformers.logging.set_verbosity_error()

good_sample = 0
stuff = []
for eval_sample in tqdm(eval_dataset):
  eval_prompt = eval_sample['prompt']
  #eval_completion = eval_sample['completion']
  eval_result = generator(eval_sample['prompt'], do_sample=False, max_new_tokens=200)
  #print("eval result is", eval_result)
  generated_text = eval_result[0]['generated_text']
  #print(generated_text[len(eval_prompt):])
  stuff.append({"prompt": eval_prompt, "completion": eval_sample['completion'], "prediction" : generated_text[len(eval_prompt):]})
  if generated_text[len(eval_prompt):] == eval_sample['completion']:
    good_sample += 1
  print(generated_text[len(eval_prompt):])

print("/n")
print("/n")
print("/n")

print("num good samples: ", good_sample)
print(stuff)


  0%|          | 2/452 [00:00<03:06,  2.42it/s]

are feedback cycles correspondence that exists between a user's actions in a system and the system's output back to the user?



  1%|          | 4/452 [00:01<03:10,  2.35it/s]

can you please detail the differences between configuration design in Lesson 21 and the method of constrained propagation from Lesson 20?
Where is the syllabus?



  1%|▏         | 6/452 [00:02<03:16,  2.27it/s]

...

define parsimony


100%|█████████▉| 450/452 [02:49<00:00,  3.69it/s]

Are analogies central to human reasoning


100%|██████████| 452/452 [02:50<00:00,  2.65it/s]

In the context of this class, can you discuss a bit about feedback and learning, specifically direct manipulation and invisible interfaces
USER: 
/n
/n
/n
num good samples:  312
[{'prompt': '[Course]HCI\n[PastChat]f different variables on the outcome measures, enabling precise comparisons between experimental conditions.\\n\\n2. **Statistical Analysis**: Quantitative data from formal experiments can be analyzed using statistical methods to determine the significance of differences between groups or conditions. Statistical analysis helps researchers assess the strength of relationships, identify patterns, and draw conclusions based on the data collected during the experiment.\\n\\n3. **Objective Measurements**: Quantitative data provides objective measurements that are not influenced by subjective biases or interpretations. By using standardized measurement tools and data collection methods, researchers can ensure the reliability and validity of the data, enhancing the credibility of th

In [ ]:
print(len(eval_dataset))



452


In [ ]:
  num_good_samples = 0
  for one_stuff in stuff:
    if(one_stuff['prediction'] == one_stuff['completion']):
      num_good_samples += 1
    else:
      print(one_stuff['completion'])
  print(num_good_samples)
  print(len(stuff))

What Python example code did I ask for previously?
Learning by recording cases
true or false: constraint propagation is concerned with whether sentences make semantic sense, not grammatical sense
USER: 
is inductive reasoning use modus ponens
Is analogical reasoning concerned with the relationships between objects
What are subactions defined by?
whereas classification trees are only useful for prototypical concepts, discrimination trees in case based reasoning can be used for exemplar concepts
Is configuration a design task where variables are given values to fulfill certain rules
can you reiterate the late policy for me 
True or false, problem reduction relies on the ability to reduce a problem to smaller subproblems, each of which must be addressed by the same problem-solving method
what module can I find these cognitive roles in
USER: 
USER: 'Which of the following approaches to design is correctly defined? Group of answer choices (A) User-Centered Design: Design that considers the 

In [ ]:
!pip install vllm
from vllm import LLM

llm = LLM(model=output_directory)
outputs = llm.generate("Hello, my name is")

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.6/394.6 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 110.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 127.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.5/31.5 MB 75.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 124.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 128.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23

INFO 06-14 18:07:53 [__init__.py:244] Automatically detected platform cuda.


ImportError: /usr/local/lib/python3.11/dist-packages/vllm/_C.abi3.so: undefined symbol: _ZN3c106ivalue14ConstantString6createENSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE